In [ ]:
#default_exp model

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# CNN


> This module contains all the code for training and extracting features from CNNs.

In [ ]:
#export
from fastai.vision import *
from fastai.metrics import error_rate
import torch
import torch.nn as nn
from torchvision import models
from torch.autograd import Variable
import cv2
from nbdev.showdoc import *
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader
import torchvision

In [ ]:
# export
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import transforms

from pytorch_lightning import Trainer

class SimCLRDataset(Dataset):
    def __init__(self, dataset):
        """Initialize a wrapper of a generic image classification dataset for SimCLR training.

        Args:
            dataset (torch.utils.data.Dataset): an image PyTorch dataset - when iterating over it
                it should return something of the form (image) or (image, label).
        """
        self.dataset = dataset

    def __getitem__(self, index):
        dataset_item = self.dataset[index]
        if type(dataset_item) is tuple:
            image = dataset_item[0]
        else:
            image = dataset_item
        return image, image

    def __len__(self):
        return len(self.dataset)

    @staticmethod
    def mixup(x, alpha=0.4):
        batch_size = x.size()[0] // 2
        if alpha > 0:
            lam = np.random.beta(alpha, alpha, batch_size)
            lam = np.concatenate(
                [lam[:, None], 1 - lam[:, None]], 1
            ).max(1)[:, None, None, None]
            lam = torch.from_numpy(lam).float()
            if torch.cuda.is_available():
                lam = lam.cuda()
        else:
            lam = 1.
        # This is SimCLR specific - we want to use the same mixing for the augmented pairs
        lam = torch.cat([lam, lam])
        index = torch.randperm(batch_size)
        # This is SimCLR specific - we want to use the same permutation on the augmented pairs
        index = torch.cat([index, batch_size + index])
        if torch.cuda.is_available():
            index = index.cuda()
        mixed_x = lam * x + (1 - lam) * x[index, :]

        return mixed_x, lam


def imagenet_normalize_transform():
    return transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])


def get_train_transforms(size=224, color_jitter_prob=0.8, grayscale_prob=0.2):
    color_jitter = transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)
    return transforms.Compose([
        transforms.RandomResizedCrop(size=(size, size)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([color_jitter], p=color_jitter_prob),
        transforms.RandomGrayscale(p=grayscale_prob),
        transforms.ToTensor(),
        imagenet_normalize_transform()
    ])


def get_val_transforms(size=224):
    return transforms.Compose([
        transforms.Resize(size=(size, size)),
        transforms.ToTensor(),
        imagenet_normalize_transform()
    ])


import torch.nn as nn


class NTXEntCriterion(nn.Module):
    """Normalized, temperature-scaled cross-entropy criterion, as suggested in the SimCLR paper.

    Parameters:
        temperature (float, optional): temperature to scale the confidences. Defaults to 0.5.
    """
    criterion = nn.CrossEntropyLoss(reduction="sum")
    similarity = nn.CosineSimilarity(dim=2)

    def __init__(self, temperature=0.5):
        super(NTXEntCriterion, self).__init__()
        self.temperature = temperature
        self.batch_size = None
        self.mask = None

    def mask_correlated_samples(self, batch_size):
        """Masks examples in a batch and it's augmented pair for computing the valid summands for
            the criterion.

        Args:
            batch_size (int): batch size of the individual batch (not including it's augmented pair)

        Returns:
            torch.Tensor: a mask (tensor of 0s and 1s), where 1s indicates a pair of examples in a
                batch that will contribute to the overall batch loss
        """
        mask = torch.ones((batch_size * 2, batch_size * 2), dtype=bool)
        mask = mask.fill_diagonal_(0)
        for i in range(batch_size):
            mask[i, batch_size + i] = 0
            mask[batch_size + i, i] = 0
        return mask

    def compute_similarities(self, z_i, z_j, temperature):
        """Computes the similarities between two projections `z_i` and `z_j`, scaling based on
            `temperature`.

        Args:
            z_i (torch.Tensor): projection of a batch
            z_j (torch.Tensor): projection of the augmented pair for the batch
            temperature (float): temperature to scale the similarity by

        Returns:
            torch.Tensor: tensor of similarities for the positive and negative pairs
        """
        batch_size = len(z_i)
        mask = self.mask_correlated_samples(batch_size)

        p1 = torch.cat((z_i, z_j), dim=0)
        sim = self.similarity(p1.unsqueeze(1), p1.unsqueeze(0)) / temperature

        sim_i_j = torch.diag(sim, batch_size)
        sim_j_i = torch.diag(sim, -batch_size)

        positive_samples = torch.cat((sim_i_j, sim_j_i), dim=0).reshape(
            batch_size * 2, 1
        )
        negative_samples = sim[mask].reshape(batch_size * 2, -1)

        logits = torch.cat((positive_samples, negative_samples), dim=1)
        return logits

    def forward(self, z):
        """Computes the loss for a batch and its augmented pair.

        Args:
            z (torch.Tensor): tensor of a batch and it's augmented pair, concatenated

        Returns:
            torch.Tensor: loss for the given batch
        """
        double_batch_size = len(z)
        batch_size = double_batch_size // 2
        z_i, z_j = z[:double_batch_size // 2], z[double_batch_size // 2:]
        if self.batch_size is None or batch_size != self.batch_size:
            self.batch_size = batch_size
            self.mask = None

        if self.mask is None:
            self.mask = self.mask_correlated_samples(self.batch_size)

        logits = self.compute_similarities(z_i, z_j, self.temperature)
        labels = torch.zeros(self.batch_size * 2).long()
        logits, labels = logits.to(z.device), labels.to(z.device)
        loss = self.criterion(logits, labels)
        loss /= 2 * self.batch_size
        return loss

class SimCLRModel(LightningModule):
    """SimCLR training network for a generic torchvision model (restricted to `allowed_models`). """

    allowed_models = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']
    allowed_datasets = ['CIFAR10', 'CIFAR100', 'STL10', 'SVHN']

    def __init__(
        self, model_name='resnet18', pretrained=True, projection_dim=64, temperature=0.5,
        batch_size=128, image_size=224, save_hparams=True
    ):
        super().__init__()
        layers = list(getattr(torchvision.models, model_name)(pretrained=pretrained).children())
        self.model = nn.Sequential(*layers[:-1])
        self.projection_head = nn.Linear(layers[-1].in_features, projection_dim)
        self.loss = NTXEntCriterion(temperature=temperature)
        self.batch_size = batch_size
        self.image_size = image_size
        self.prepare_data()

    def forward(self, x):
        out = self.model(x)
        out = out.view(x.size(0), -1)
        out = self.projection_head(out)
        return out

    def training_step(self, batch, batch_idx):
        projections = self(batch)
        loss = self.loss(projections)
        tensorboard_logs = {'train_loss': loss}
        self.logger.scalar('loss', loss)
        return {'loss': loss, 'log': tensorboard_logs}

    def training_epoch_end(self, outputs):
        loss_mean = torch.stack([x['loss'] for x in outputs]).mean()
        return {'train_loss': loss_mean}

    def configure_optimizers(self):
        return torch.optim.Adam([
            {'params': self.model.parameters(), 'lr': 0.00001},
            {'params': self.projection_head.parameters(), 'lr': 0.001}
        ])

    def prepare_data(self):
        train_transforms, val_transforms = (
            get_train_transforms(size=self.image_size),
            get_val_transforms(size=self.image_size)
        )
        train_dataset = torchvision.datasets.ImageFolder(
            '/tf/data/combined', transform = train_transforms)
        self.train_dataset = SimCLRDataset(train_dataset)

    def collate_fn(self, batch):
        return torch.cat([torch.stack([b[0] for b in batch]), torch.stack([b[1] for b in batch])])

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset, batch_size=self.batch_size, num_workers=64, shuffle=True,
            collate_fn=self.collate_fn
        )

In [ ]:
#export
class Hook():
    "Create a hook on `m` with `hook_func`."
    def __init__(self, m:nn.Module, hook_func:HookFunc, is_forward:bool=True, detach:bool=True):
        self.hook_func,self.detach,self.stored = hook_func,detach,None
        f = m.register_forward_hook if is_forward else m.register_backward_hook
        self.hook = f(self.hook_fn)
        self.removed = False

    def hook_fn(self, module:nn.Module, input:Tensors, output:Tensors):
        "Applies `hook_func` to `module`, `input`, `output`."
        if self.detach:
            input  = (o.detach() for o in input ) if is_listy(input ) else input.detach()
            output = (o.detach() for o in output) if is_listy(output) else output.detach()
        self.stored = self.hook_func(module, input, output)

    def remove(self):
        "Remove the hook from the model."
        if not self.removed:
            self.hook.remove()
            self.removed=True

    def __enter__(self, *args): return self
    def __exit__(self, *args): self.remove()

In [ ]:
#export
def get_output(module, input_value, output):
    return output.flatten(1)

def get_input(module, input_value, output):
    return list(input_value)[0]

def get_named_module_from_model(model, name):
    for n, m in model.named_modules():
        if n == name:
            return m
    return None

In [ ]:
#export
def trainPetsModel():
    '''Fine tunes a pretrained ResNet50 model to detect pet breeds 
    and returns the resulting model in learn as well as the last layer
    in linear_output_layer for subsequent feature extraction.'''
    
    bs = 16
    np.random.seed(2)
    
    path = untar_data(URLs.PETS)
    path_img = path/'images'
    fnames = get_image_files(path_img)
    pat = pat = r'/([^/]+)_\d+.jpg$'
    data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), size=224, bs=bs
                                  ).normalize(imagenet_stats)
    learn = cnn_learner(data, models.resnet34, metrics=error_rate)
    learn.fit_one_cycle(4)

    linear_output_layer = get_named_module_from_model(learn.model, '1.4')
    return learn, linear_output_layer

In [ ]:
#export
def getFeaturesResNet50(img):
    '''Directly gets a feature vector from the last layer of ResNet50
    without any fine tuning.'''
    
    model = models.resnet50(pretrained=True)
    modules = list(model.children())[:-1]
    resNet50 = nn.Sequential(*modules)
    imgVar = Variable(img)
    featuresVar = resNet50.eval()(imgVar)
    features = featuresVar.data
    
    return features
    

In [ ]:
#export
def trainAsModel():
    '''Fine tunes a pretrained ResNet50 model to detect Google Play store  
    categories of Android screenshots and returns the resulting model in learn 
    as well as the last layer in linear_output_layer for subsequent feature extraction.'''
    
    bs = 16
    np.random.seed(2)
    
    path = "/tf/GP-Labeled-Dataset/"
    data = ImageDataBunch.from_folder(path, ds_tfms=get_transforms(), size=299, bs=bs).normalize(imagenet_stats)
    learn = cnn_learner(data, models.resnet50, metrics=error_rate)
    learn.load('/tf/models/Android_ResNet50_16Epochs')
    
    linear_output_layer = get_named_module_from_model(learn.model, '1.4')
    return learn, linear_output_layer

In [ ]:
#export
class Extractor():
    
    '''Low level implementation of Extractor class for non-layered models only.'''
    
    def __init__(self, learn, linear_output_layer, model, isFT):
        self.models = {'resnet50'}
        self.learn = learn
        self.linear_output_layer = linear_output_layer
        self.model = model
        self.isFT = isFT
        
        if self.model not in self.models:
            raise Exception("Invalid architecture: " + self.model)
    
    def getFeatures(self, img):
        if self.model == 'resnet50':
            resized_img = cv2.resize(img, (224, 224))
        final_img = np.expand_dims(np.transpose(resized_img, [2, 0, 1]), axis=0)
        tensor = torch.from_numpy(final_img)
        
        img_repr = []
        if self.isFT:
            floatTensor = tensor.type(torch.cuda.FloatTensor)
            with Hook(self.linear_output_layer, get_output, True, True) as hook:
                result = self.learn.model.eval()(floatTensor)
                img_repr = hook.stored.cpu().numpy()
        else:
            floatTensor = tensor.type(torch.FloatTensor)
            img_repr = getFeaturesResNet50(floatTensor).numpy().reshape(1, -1)
        
        return img_repr
    

In [ ]:
#export
class LayeredExtractor():
    
    '''Low level implementation of Extractor class for layered models only.'''
    
    def __init__(self):
        self.features = []
        
    def hook_fn(self, module, input, output):
        output_flat = output.detach().flatten()
        self.features = np.hstack((self.features, output_flat))

    def getFeatures(self, img):
        self.features = []
        
        resized_img = cv2.resize(img, (224, 224))
        final_img = np.expand_dims(np.transpose(resized_img, [2, 0, 1]), axis=0)
        tensor = torch.from_numpy(final_img)
        floatTensor = tensor.type(torch.FloatTensor)
        
        model = models.resnet50(pretrained=True)
        for n, m in model.named_modules():
            if 'pool' in n:
                m.register_forward_hook(self.hook_fn)
        model.eval()(floatTensor)
        self.features = self.features.reshape(1, -1)
        return self.features
    

In [ ]:
#export
def createExtractor(learn, linear_output_layer, model, isFT):
    return Extractor(learn, linear_output_layer, model, isFT)
def createLayeredExtractor():
    return LayeredExtractor()

In [2]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_prep.ipynb.
Converted 01_features.ipynb.
Converted 02_eval.ipynb.
Converted 03_cnn.ipynb.
This cell doesn't have an export destination and was ignored:
 
Converted 04_experiments.ipynb.
Converted 05_model.ipynb.
Converted SimCLR.old.ipynb.
Converted [Scratch 1] Tango SimCLR.ipynb.
Converted [Scratch 2] Tango SimCLR.ipynb.
Converted index.ipynb.
Converted lesson1-pets.ipynb.
